In [1]:
import cv2
import numpy as np
import time
import itertools
%matplotlib inline

# Simple Video Capture and Analysis
In the below example, you can use the windows from highgui to change what part of the code is getting implemented. Press the digits 1-9 to change which conditional is getting processed.

In [3]:
section = 0

# setup some windows for viewing
cv2.namedWindow("demowin1")
cv2.namedWindow("demowin2")
cv2.startWindowThread()

# open the video card for capture
vc = cv2.VideoCapture(0)

if vc.isOpened():  # try to get the first frame
    print ("vc opened, getting first frame")
    rval, frame = vc.read()
    # this will likely fail the first time
    # the webcam often needs some time to open fully
    key = 0
else:
    print ("vc not open, exiting")
    key = 27

while key != 27 and vc.isOpened():  # the escape key and the capture device is open
    rval, frame = vc.read()
    key = cv2.waitKey(10)

    # interpret the input key, on top number line of keyboard
    if ord('0') <= key <= ord('7'):
        section = key - ord('0')  # press 0, 1, 2, ... 9

    if rval and frame is not None:
        frame = cv2.pyrDown(frame)  # make smaller immediately

        if section == 0:
            pass  # just display the WebCam image

        elif section == 1:
            # get the width and the height and the depth
            h, w, d = frame.shape
            i = int(h / 2)
            j = int(w / 2)

            # slow access
            t = time.time()
            for iterate in range(0, 10000):
                b = frame[i, j, 0]
                g = frame[i, j, 1]
                r = frame[i, j, 2]
            str1 = "B1:%d, G1:%d, R1:%d, time=%.5f" % (b, g, r, time.time() - t)

            # speed up access using NumPy accelerations
            t = time.time()
            for iterate in range(0, 10000):
                b = frame.item(i, j, 0)
                g = frame.item(i, j, 1)
                r = frame.item(i, j, 2)
            str2 = "B2:%d, G2:%d, R2:%d, time=%.5f" % (b, g, r, time.time() - t)

            # set the value
            for x in itertools.product(range(i - 10, i + 10), range(j - 10, j + 10)):
                frame.itemset(x[0], x[1], 0, 255)  # blue value

            cv2.putText(frame, str1, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 0])
            cv2.putText(frame, str2, (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 255])

        elif section == 2:
            # convert to gray
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow("demowin2", gray)

        elif section == 3:
            # convert to HSV and then grab the Hue component
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            singleChannel = cv2.split(hsv)[0]
            cv2.imshow("demowin2", singleChannel)

        elif section == 4:
            # scaling down an image (can speed things up if your frame rate gets too slow)
            small_frame = cv2.pyrDown(frame)
            cv2.imshow("demowin2", small_frame)

        elif section == 5:
            # smoothing with a filter
            kernel = cv2.getGaussianKernel(25, 3)
            smooth_frame = cv2.filter2D(frame, -1, kernel)
            cv2.imshow("demowin2", smooth_frame)

        elif section == 6:
            # Requires a single-channel image:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Reduce the image to the edges detected:
            gray = cv2.Canny(gray, 50, 100, 3)  # Arbitrarily chosen parameters. See documentation for the meaning
            cv2.imshow("demowin2", gray)

        elif section == 7:
            # Hough Circles:  http://en.wikipedia.org/wiki/Hough_transform

            # Requires a single-channel image:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Want to smooth it to get less noise
            kernel = cv2.getGaussianKernel(9, 1)
            gray = cv2.filter2D(gray, -1, kernel)

            # Detect the circles in the image
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 4, minDist=800,
                                       param1=300, param2=100, minRadius=30, maxRadius=70)
            if circles is not None:
                circles = np.uint16(np.around(circles[0]))
                # Iterate through the list of circles found by cvHoughCircles()
                for c in circles:
                    # It has the format: [center_x, center_y, radius]
                    # lets draw them
                    center = (c[0], c[1])  # center of the circle
                    rad = c[2]  # radius of the circle
                    cv2.circle(frame, center, rad, (0, 255, 0), 1)

                    # # There's lots of drawing commands you can use!
                    # Here is some c++ code to get you on the right track:
                    # CvFont font;
                    # cvInitFont(&font, CV_FONT_HERSHEY_SIMPLEX, 1, 1, 0.0, 1, 8);
                    # cvCircle( frame, cvPoint(cvRound(p[0]),cvRound(p[1])), cvRound(p[2]), CV_RGB(255,0,0), 3, 8, 0 );
                    # cvPutText( frame, "Circle", cvPoint(cvRound(p[0]),cvRound(p[1])), &font, CV_RGB(255,0,0) );

        # we have an image to process
        cv2.putText(frame, "Section = " + str(section), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 0])
        cv2.imshow("demowin1", frame)

print('releasing...')
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
print('Done')

vc opened, getting first frame
releasing...
Done


# Processing Color
This will isolate similar hues in the image. You can adjust the threshold you are using by pressing `u` and `d` to increase or decrease the value of the threshold. 

In [2]:
import cv2

# setup some windows for viewing
cv2.namedWindow("WebCam")
cv2.namedWindow("Hue")
cv2.startWindowThread()

vc = cv2.VideoCapture(0)

if vc.isOpened():
    print ("vc is open!!")
    return_value, frame = vc.read()

h_range = 30
key = -1
while key != 27 and vc.isOpened():
    return_value, frame = vc.read()

    if frame is not None:

        height, width, depth = frame.shape

        frame = cv2.pyrDown(frame)

        hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv_img)

        h = cv2.inRange(h, h_range, h_range+10)
        h_binary = h.copy()
        _, contours, hierarchy = cv2.findContours(h_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        if contours is not None and len(contours) > 0:
            for cnt in contours:
                area = cv2.contourArea(cnt)
                # bb   = cv2.boundingRect(cnt)

                pt = tuple(cnt[0][0])
                if area > 100:
                    cv2.putText(h, str(area), pt, cv2.FONT_HERSHEY_SIMPLEX, 0.25, 255)

        cv2.putText(h, str(h_range), (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, 128)
        cv2.imshow("WebCam", frame)
        cv2.imshow("Hue", h)

    key = cv2.waitKey(10)
    if key == ord('u'):
        h_range += 1
    elif key == ord('d'):
        h_range -= 1

print('releasing...')
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
print('Done')

vc is open!!
releasing...
Done


# Processing Skin and Understanding Hands
This will isolate skin in an image based on a calibration of the color. Hold you hand over the "squares" in the "input" image window and press `c` to calibrate the software. Once calibrated, the system will attempt to segment the skin in the image (based on the color variation it observed from your calibration image). Then it will attempt to find the convex hull and convex deformations of the skin to estimate likely locations of finger tips. 

To perform a new calibration, just press `u` to undo the previsou calibration. Then you can repeat the above steps as needed. 

In [4]:
ESC_KEY = 27  # for allowing the user to exit the program
C_KEY = ord('c')
U_KEY = ord('u')

import cv2
import cvhelper  # some helper files written by eclarson
# setup some windows for viewing
cv2.namedWindow("input")
cv2.namedWindow("output")
cv2.startWindowThread()

def set_patches(img_in):
    height, width, depth = img_in.shape

    patches = [(width / 2, 3 * height / 4), (width / 2, height / 2), (width / 2, height / 4),
               (width / 2 + 45, 5 * height / 8), (width / 2 - 10, 5 * height / 8 + 10),
               (width / 2 + 25, 5 * height / 8 - 20), (width / 2 - 35, 5 * height / 8)]

    return patches


cap = cv2.VideoCapture(0)  # video capture on default input stream
if cap.isOpened():
    print ("Video opened, getting first frame")
    ret, img = cap.read()  # get a frame from the video card

k = -1
calibrated = False
while cap.isOpened():
    ret, img = cap.read()  # get a frame from the video card
    if ret and (img is not None):
        img = cvhelper.pre_process(img)
        hsv_img = cvhelper.convert_color(img)
        hand_patches = set_patches(img)

        if k == C_KEY:
            # user initiated a calibrate command
            median_values = cvhelper.find_median_colors(hsv_img, hand_patches, 10)
            calibrated = True
            print (median_values)

        elif k == U_KEY:
            calibrated = False

        if not calibrated:
            # show the user where to place their hand
            img = cvhelper.set_img_boxes(img, hand_patches, 10)
        else:
            # process where we think the hand is
            skin_img = cvhelper.segment_skin(hsv_img, median_values)
            cv2.imshow('skin', skin_img)
            hsv_img = cvhelper.find_contours(skin_img)

        cv2.imshow('output', hsv_img)
        cv2.imshow('input', img)

    # get possible text input from the open figures
    # if a user presses a key, it will show up here
    k = cv2.waitKey(10)
    if k == ESC_KEY:
        break

print('releasing...')
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
print('Done')

Video opened, getting first frame
[(163.0, 45.0, 148.0), (159.0, 45.0, 163.5), (170.0, 102.0, 120.0), (156.0, 44.0, 165.0), (164.0, 47.0, 143.0), (166.0, 67.0, 139.0), (158.0, 42.0, 156.0)]
[(175.0, 95.5, 127.0), (174.0, 87.0, 126.0), (139.0, 33.0, 205.0), (137.0, 35.0, 162.0), (168.0, 63.0, 143.0), (166.0, 62.0, 131.0), (165.0, 53.0, 146.0)]
releasing...
Done
